# Exercise Fourteen: Project Design Starter

In this exercise, you'll be planning out a complex project. You'll draw in some code, but focus on commenting to describe your project structure. The sample document below will guide you through organizing and annotating your project design. The primary components you'll include are:

- **Dependencies:** What modules will your project need?
- **Collection:** Where is your data coming from?
- **Processing:** How will you format and process your data?
- **Analysis:** What techniques will you use to understand your data?
- **Visualization:** How will you visualize and explore your data?

Don't worry if you aren't exactly certain how you would implement everything - this should be a starting point for a larger research study, but it doesn't need to be a complete, functional workflow. Aim for a "good enough" starting point that you can reference and extend for future work. 

Note where you have something working, and where it's broken or in progress.

## Project Overview

Replace this text with a short description of what your envisioned project design will accomplish. Include your research question and goals for this analysis.

## Dependencies

Add the import code for every dependency of your project: for instance, if you are collecting data, you might import Tweepy or BeautifulSoup. If you're working with a file of folders, import os. Most projects will require Pandas, along with appropriate processing and visualization libraries. In the comments, explain briefly why you are including each library (as shown in the example below.) 

In [10]:
# Importing Pandas to handle collected Twitter data (example comment)
import pandas as pd
import os
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.io import output_notebook

## Collection

Describe your data collection scope and process briefly, and include an example of how you might collect your data drawing on our other projects. For example, if this workflow will collect Twitter data from a stream, you might revisit that demo, copy the stream, and adjust the hashtag.

In [11]:
# Collect data using a Tweepy stream (example annotation)
# (Copy and modify code from other exercises to prototype this goal)

abmc_df = pd.read_csv('all_abmc_wwi.csv', delimiter=",")

abmc_df


FileNotFoundError: [Errno 2] File all_abmc_wwi.csv does not exist: 'all_abmc_wwi.csv'

## Processing

After your data has been collected or imported, store it in a format that works for your purposes. This can vary: for Twitter analysis, it might be a Pandas dataframe, while for text, you might build a document term matrix.

In [ ]:
# Store Twitter data using Pandas with appropriate column names (example comment)
# (Copy and modify code from other exercises to prototype this goal)

## Analysis

Think across all of the methods we've tried this semester - what combination would be most helpful for your goals? Include code sections for each method you think is important. In most cases, a combination will be most revealing: for instance, you might employ several different textual analysis frameworks on a set of documents. Use at least two distinctly different methods of analysis.

In [ ]:
# Compare several account outputs using PCA (example comment)
# (Copy and modify code from other exercises to prototype this goal)

## Visualization

Finally, think about the visualizations that would be most useful to sharing and exploring your data. Consider both static and dynamic approaches from the different libraries we've worked with this semester. Include at least two preliminary visualizations.

In [ ]:
# Build a wordcloud of term distributions in each document (example comment)
# (Copy and modify code from other exercises to prototype this goal)